In [24]:
import numpy as np
import struct
import time

def hexdump(bytes_input, width=16):
    current = 0
    end = len(bytes_input)
    result = ""
    while current < end:
        result += "%08x: " % current
        byte_slice = bytes_input[current : current + width]
        for b in byte_slice:
            result += "%02x " % b
        for _ in range(width - len(byte_slice)):
            result += " " * 3
        result += " "
        for b in byte_slice:
            if (b >= 32) and (b < 127):
                result += chr(b)
            else:
                result += "."
        result += "\n"
        current += width
    return result

In [42]:
with open("ibeacon.dat", "rb") as f:
    data = f.read()

print(hexdump(data))

00000000: aa d6 be 89 8e 40 32 05 a2 17 6e 3d 71 02 01 06  .....@2...n=q...
00000010: 1a ff 59 00 02 15 01 12 23 34 45 56 67 78 89 9a  ..Y.....#4EVgx..
00000020: ab bc cd de ef f0 00 00 00 00 c1 0d 09 54 65 73  .............Tes
00000030: 74 20 69 42 65 61 63 6f 6e 3f fa 68              t iBeacon?.h



In [44]:
PCAP_MAGIC_NUMBER = b"\xd4\xc3\xb2\xa1"
LINKTYPE_BLUETOOTH_LE_LL = 251
LINKTYPE_BLUETOOTH_LE_LL_WITH_PHDR = 256

def pcap_global_header():
    header =  PCAP_MAGIC_NUMBER
    header += struct.pack("HH", 2, 4)
    header += b"\x00" * 8
    header += struct.pack("I", 0xffff)
    header += struct.pack("I", LINKTYPE_BLUETOOTH_LE_LL_WITH_PHDR)
    return header

def pcap_packet_header(length=0):
    timestamp = time.time()
    seconds = int(timestamp)
    useconds = int((timestamp - seconds) * 1000000)
    header = struct.pack("IIII", seconds, useconds, length, length)
    return header

def pcap_packet_data(pkt=b""):
    data = b"\x0c\xff\xff\x00\xd6\xbe\x89\x8e\x37\x3c"
    data += pkt
    return data

def pcap_ble_export(ble_pkts):
    buf = b""
    global_header = pcap_global_header()
    buf += global_header
    for pkt in ble_pkts:
        # remove preamble
        pkt = pkt[1:]
        pkt_header = pcap_packet_header(len(pkt) + 10)
        pkt_data = pcap_packet_data(pkt)
        buf += pkt_header + pkt_data
    return buf

pcap_content = pcap_ble_export([data, data, data])
print(hexdump(pcap_content))

with open("ble_pcap_demo.cap", "wb") as f:
    f.write(pcap_content)

00000000: d4 c3 b2 a1 02 00 04 00 00 00 00 00 00 00 00 00  ................
00000010: ff ff 00 00 00 01 00 00 d0 7b ce 63 49 97 05 00  .........{.cI...
00000020: 45 00 00 00 45 00 00 00 0c ff ff 00 d6 be 89 8e  E...E...........
00000030: 37 3c d6 be 89 8e 40 32 05 a2 17 6e 3d 71 02 01  7<....@2...n=q..
00000040: 06 1a ff 59 00 02 15 01 12 23 34 45 56 67 78 89  ...Y.....#4EVgx.
00000050: 9a ab bc cd de ef f0 00 00 00 00 c1 0d 09 54 65  ..............Te
00000060: 73 74 20 69 42 65 61 63 6f 6e 3f fa 68 d0 7b ce  st iBeacon?.h.{.
00000070: 63 4b 97 05 00 45 00 00 00 45 00 00 00 0c ff ff  cK...E...E......
00000080: 00 d6 be 89 8e 37 3c d6 be 89 8e 40 32 05 a2 17  .....7<....@2...
00000090: 6e 3d 71 02 01 06 1a ff 59 00 02 15 01 12 23 34  n=q.....Y.....#4
000000a0: 45 56 67 78 89 9a ab bc cd de ef f0 00 00 00 00  EVgx............
000000b0: c1 0d 09 54 65 73 74 20 69 42 65 61 63 6f 6e 3f  ...Test iBeacon?
000000c0: fa 68 d0 7b ce 63 4c 97 05 00 45 00 00 00 45 00  .h.{.cL...E...E.
000000d0: 00